In [1]:
# Configuração do ambiente
import os
from dotenv import load_dotenv

load_dotenv()
#os.environ['SPARK_VERSION'] = os.getenv('SPARK_VERSION')
os.environ['PYSPARK_PYTHON'] = os.getenv('PYSPARK_PYTHON') 
os.environ['SPARK_HOME'] = os.getenv('SPARK_HOME')
os.environ['HADOOP_HOME'] = os.getenv('HADOOP_HOME')
os.environ['JAVA_HOME'] = os.getenv('JAVA_HOME')

In [2]:
# Importação de bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pydeequ
from pydeequ.analyzers import *

In [3]:
# Configuração spark session 
spark = (
    SparkSession.builder
    .config('spark.ui.port', '4050')
    .config('spark.jars.packages', pydeequ.deequ_maven_coord)
    .config('spark.jars.excludes', pydeequ.f2j_maven_coord)
    .appName('SparkSQL')
    .getOrCreate()
)

In [4]:
# Configuração spark UI
ui_url = spark.sparkContext.uiWebUrl
print("URL da Spark UI:", ui_url)

URL da Spark UI: http://DESKTOP-E8QPPRO:4050


In [5]:
# Ingestão dos dados
PATH_JSON = '../data/case_final.json'

# Configuração de schema
schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])


schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', TimestampType()),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('fraude', IntegerType())
])

# Leitura
df = spark.read.json(
    PATH_JSON,
    schema=schema_base_pix,
    timestampFormat='yyyy-MM-dd HH:mm:ss'
)

# Transformação destinatário e remetente para formato colunar
df = df.withColumn(
    'destinatario_nome', col('destinatario').getField('nome')
).withColumn(
    'destinatario_banco', col('destinatario').getField('banco')
).withColumn(
    'destinatario_tipo', col('destinatario').getField('tipo')
).withColumn(
    'remetente_nome', col('remetente').getField('nome')
).withColumn(
    'remetente_banco', col('remetente').getField('banco')
).withColumn(
    'remetente_tipo', col('remetente').getField('tipo')
).drop('remetente', 'destinatario')

df.show()

+------------+------------------+-------------------+---------+-------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+
|id_transacao|             valor|   transaction_date|chave_pix|    categoria|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|    remetente_nome|remetente_banco|remetente_tipo|
+------------+------------------+-------------------+---------+-------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+
|        1000|            588.08|2021-07-16 05:00:55|aleatoria|       outros|     0|         Calebe Melo|             Caixa|               PF|Jonathan Gonsalves|            BTG|            PF|
|        1001|           80682.5|2022-04-20 12:34:01|  celular|transferencia|     1|  Davi Lucas Pereira|             Caixa|               PJ|Jonathan Gonsalves|            BTG|            PF|
|        1002|             549.9|20

In [6]:
df.describe().show()

+-------+-----------------+------------------+---------+-----------+------------------+-----------------+------------------+-----------------+------------------+---------------+--------------+
|summary|     id_transacao|             valor|chave_pix|  categoria|            fraude|destinatario_nome|destinatario_banco|destinatario_tipo|    remetente_nome|remetente_banco|remetente_tipo|
+-------+-----------------+------------------+---------+-----------+------------------+-----------------+------------------+-----------------+------------------+---------------+--------------+
|  count|           100000|            100000|   100000|     100000|            100000|           100000|            100000|           100000|            100000|         100000|        100000|
|   mean|          50999.5|10303.358732200059|     null|       null|           0.15367|             null|              null|             null|              null|           null|          null|
| stddev|28867.65779668774| 20874.9

In [7]:
df.createOrReplaceTempView('transacoes_pix')

In [23]:
# Bancos com maior número de transferências
spark.sql(
    '''
    SELECT 
        destinatario_banco, count(destinatario_banco) as transferencias
    FROM
        transacoes_pix
    GROUP BY
        destinatario_banco
    ORDER BY
        transferencias DESC
    LIMIT 10
    '''
).show()

+------------------+--------------+
|destinatario_banco|transferencias|
+------------------+--------------+
|                XP|         14401|
|               BTG|         14390|
|            Nubank|         14297|
|              Itau|         14281|
|             Caixa|         14240|
|                C6|         14204|
|          Bradesco|         14187|
+------------------+--------------+



In [40]:
# Média de valor das transferências por período
spark.sql(
    '''
    SELECT
        DATE_FORMAT(transaction_date, 'yyyy-MM') AS periodo,
        AVG(valor) AS valor_medio
    FROM
        transacoes_pix
    GROUP BY
        periodo
    ORDER BY
        periodo ASC
    '''
).show(df.count())



+-------+------------------+
|periodo|       valor_medio|
+-------+------------------+
|2021-01| 9961.148306418205|
|2021-02|10254.565595677386|
|2021-03|10226.794436468064|
|2021-04| 10174.00809899041|
|2021-05|10009.756290959804|
|2021-06|10185.070611137333|
|2021-07|10171.928728059656|
|2021-08| 11072.60574823528|
|2021-09| 10280.04686523439|
|2021-10|11021.997309872451|
|2021-11|10227.241328395072|
|2021-12|  10480.7907325142|
|2022-01|10507.262373201262|
|2022-02| 10123.09668741892|
|2022-03| 10558.32449918284|
|2022-04|10355.513566212305|
|2022-05|10297.858686630503|
|2022-06|10172.860004888767|
|2022-07|10040.668731549818|
|2022-08|10797.367360454096|
|2022-09|10462.071680387418|
|2022-10| 10280.48527731492|
|2022-11| 9824.621844802343|
|2022-12| 9742.439687203792|
|2023-01|10002.365324144983|
+-------+------------------+

